#Email Security

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# for handling and parsing email files
import email
import os
import re
import nltk
from email import message_from_string
from email.message import EmailMessage

from wordcloud import WordCloud
from bs4 import BeautifulSoup
from collections import Counter
# modeling
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from google.colab import files

# Create a file upload button.
uploaded = files.upload()

Saving spam_zipped.zip to spam_zipped.zip
Saving ham_zipped.zip to ham_zipped.zip


In [ ]:
!unzip -q spam_zipped.zip -d /content/spam_emails
!unzip -q ham_zipped.zip -d /content/ham_emails

In [ ]:
!ls /content/ham_emails
!ls /content/spam_emails

main_ham
main_spam


In [ ]:
import os

def read_files(directory):
    emails = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        try:
            with open(filepath, 'r', encoding='utf-8') as file:
                emails.append(file.read())
        except UnicodeDecodeError:
            try:
                with open(filepath, 'r', encoding='latin1') as file:
                    emails.append(file.read())
            except Exception as e:
                print(f"Failed to read {filepath} with both UTF-8 and Latin1: {str(e)}")
    return emails

# Reading emails from both directories
ham_emails = read_files('/content/ham_emails/main_ham')
spam_emails = read_files('/content/spam_emails/main_spam')

# Example to print the first email from each list
if ham_emails:
    print("First ham email:", ham_emails[0])
if spam_emails:
    print("First spam email:", spam_emails[0])


First ham email: From fork-admin@xent.com  Wed Sep 25 10:24:54 2002
Return-Path: <fork-admin@xent.com>
Delivered-To: yyyy@localhost.spamassassin.taint.org
Received: from localhost (jalapeno [127.0.0.1])
	by jmason.org (Postfix) with ESMTP id 7ED4016F03
	for <jm@localhost>; Wed, 25 Sep 2002 10:24:54 +0100 (IST)
Received: from jalapeno [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for jm@localhost (single-drop); Wed, 25 Sep 2002 10:24:54 +0100 (IST)
Received: from xent.com ([64.161.22.236]) by dogma.slashnull.org
    (8.11.6/8.11.6) with ESMTP id g8P8b2C19248 for <jm@jmason.org>;
    Wed, 25 Sep 2002 09:37:02 +0100
Received: from lair.xent.com (localhost [127.0.0.1]) by xent.com (Postfix)
    with ESMTP id 6CF4D2940EB; Wed, 25 Sep 2002 01:33:09 -0700 (PDT)
Delivered-To: fork@spamassassin.taint.org
Received: from argote.ch (argote.ch [80.65.224.17]) by xent.com (Postfix)
    with ESMTP id 858F029409A for <fork@xent.com>; Wed, 25 Sep 2002 01:32:34
    -0700 (PDT)
Received: by argo

In [ ]:
hl, sl = len(ham_emails), len(spam_emails)

print("Number of ham files:", hl)
print("Number of spam files:", sl)
print("Spam to Ham ratio:", sl/hl)
print("Spam percentage: ", (sl / (sl + hl)) * 100, "%")

Number of ham files: 6951
Number of spam files: 2398
Spam to Ham ratio: 0.34498633290174074
Spam percentage:  25.64980211787357 %


In [ ]:
test_email = ham_emails[0]
test_email

"From fork-admin@xent.com  Wed Sep 25 10:24:54 2002\nReturn-Path: <fork-admin@xent.com>\nDelivered-To: yyyy@localhost.spamassassin.taint.org\nReceived: from localhost (jalapeno [127.0.0.1])\n\tby jmason.org (Postfix) with ESMTP id 7ED4016F03\n\tfor <jm@localhost>; Wed, 25 Sep 2002 10:24:54 +0100 (IST)\nReceived: from jalapeno [127.0.0.1]\n\tby localhost with IMAP (fetchmail-5.9.0)\n\tfor jm@localhost (single-drop); Wed, 25 Sep 2002 10:24:54 +0100 (IST)\nReceived: from xent.com ([64.161.22.236]) by dogma.slashnull.org\n    (8.11.6/8.11.6) with ESMTP id g8P8b2C19248 for <jm@jmason.org>;\n    Wed, 25 Sep 2002 09:37:02 +0100\nReceived: from lair.xent.com (localhost [127.0.0.1]) by xent.com (Postfix)\n    with ESMTP id 6CF4D2940EB; Wed, 25 Sep 2002 01:33:09 -0700 (PDT)\nDelivered-To: fork@spamassassin.taint.org\nReceived: from argote.ch (argote.ch [80.65.224.17]) by xent.com (Postfix)\n    with ESMTP id 858F029409A for <fork@xent.com>; Wed, 25 Sep 2002 01:32:34\n    -0700 (PDT)\nReceived: b

In [ ]:
from email import policy
from email.parser import BytesParser

def extract_email_body(filepath):
    with open(filepath, 'rb') as file:  # read the email in binary mode
        email = BytesParser(policy=policy.default).parse(file)
    if email.is_multipart():
        # Collect parts that are text/plain or text/html
        for part in email.walk():
            ctype = part.get_content_type()
            if ctype in ('text/plain', 'text/html'):
                content = part.get_payload(decode=True).decode('latin1', errors='ignore')
                return content.lower()  # Convert content to lower case
    else:
        # Handle non-multipart emails
        return email.get_payload(decode=True).decode('latin1', errors='ignore').lower()

# Example usage: Adjust the path to where your individual email files are stored
email_body = extract_email_body('/content/ham_emails/main_ham/00001.1a31cc283af0060967a233d26548a6ce')
print(email_body)

    date:        tue, 20 aug 2002 17:27:47 -0500
    from:        chris garrigues <cwg-exmh@deepeddy.com>
    message-id:  <1029882468.3116.tmda@deepeddy.vircio.com>


  | i'm hoping that all people with no additional sequences will notice are
  | purely cosmetic changes.

well, first, when exmh (the latest one with your changes) starts, i get...

can't read "flist(totalcount,unseen)": no such element in array
    while executing
"if {$flist(totalcount,$mhprofile(unseen-sequence)) > 0} {
	flaginner spool iconspool labelup
    } else {
	flaginner down icondown labeldown
    }"
    (procedure "flag_msgseen" line 3)
    invoked from within
"flag_msgseen"
    (procedure "msgseen" line 8)
    invoked from within
"msgseen $msgid"
    (procedure "msgshow" line 12)
    invoked from within
"msgshow $msgid"
    (procedure "msgchange" line 17)
    invoked from within
"msgchange 4862 show"
    invoked from within
"time [list msgchange $msgid $show"
    (procedure "msg_change" line 3)
    invoked f

In [ ]:
import os

def read_and_process_emails(directory):
    emails = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        email_body = extract_email_body(filepath)
        emails.append(email_body)  # Append the processed body to the list
    return emails

ham_emails = read_and_process_emails('/content/ham_emails/main_ham')
spam_emails = read_and_process_emails('/content/spam_emails/main_spam')


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Assuming you have lists `ham_emails` and `spam_emails` with email contents
emails = ham_emails + spam_emails
labels = ['ham'] * len(ham_emails) + ['spam'] * len(spam_emails)

# Create TF-IDF features
tfidf_vectorizer = TfidfVectorizer()
features = tfidf_vectorizer.fit_transform(emails)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)


Isolation forest

In [ ]:
# Initialize the Isolation Forest model
iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)

# Fit the model on your training data
iso_forest.fit(X_train)

# Predict anomalies on the test set
# Isolation Forest marks inliers as 1 and outliers as -1
if_predictions = iso_forest.predict(X_test)

# Convert predictions: -1 (outliers, i.e., spam) to 'spam' and 1 (inliers, i.e., ham) to 'ham'
if_predictions = ['spam' if pred == -1 else 'ham' for pred in if_predictions]

# Evaluate the model
print("Isolation Forest Classifier Report:")
print(classification_report(y_test, if_predictions))
print("Accuracy:", accuracy_score(y_test, if_predictions))

Isolation Forest Classifier Report:
              precision    recall  f1-score   support

         ham       0.74      1.00      0.85      1734
        spam       0.00      0.00      0.00       604

    accuracy                           0.74      2338
   macro avg       0.37      0.50      0.43      2338
weighted avg       0.55      0.74      0.63      2338

Accuracy: 0.7416595380667237


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Support Vector Machine

In [ ]:
from sklearn.svm import SVC

# Train a Support Vector Machine classifier
svm_classifier = SVC(kernel='linear')  # Linear kernel
svm_classifier.fit(X_train, y_train)
svm_predictions = svm_classifier.predict(X_test)
print("Support Vector Machine Classifier Report:")
print(classification_report(y_test, svm_predictions))

Support Vector Machine Classifier Report:
              precision    recall  f1-score   support

         ham       0.99      1.00      1.00      1734
        spam       1.00      0.98      0.99       604

    accuracy                           0.99      2338
   macro avg       0.99      0.99      0.99      2338
weighted avg       0.99      0.99      0.99      2338



Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Train a Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)
nb_predictions = nb_classifier.predict(X_test)
print("Naive Bayes Classifier Report:")
print(classification_report(y_test, nb_predictions))

Naive Bayes Classifier Report:
              precision    recall  f1-score   support

         ham       0.88      1.00      0.93      1734
        spam       0.98      0.61      0.75       604

    accuracy                           0.89      2338
   macro avg       0.93      0.80      0.84      2338
weighted avg       0.90      0.89      0.89      2338



#DDoS attack mitigation

Neural Network

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

In [ ]:
train_data=pd.read_csv('/content/TRAIN-DATA.csv')
test_data=pd.read_csv('/content/TEST-DATA.csv')
print("Train Data")
print(train_data.head())

Train Data
   datapath_id  version  header_length  tos  total_length  flags  offset  ttl  \
0            1        4              5    0            28      0       0  250   
1            1        4              5    0            28      0       0  250   
2            1        4              5    0            28      0       0  250   
3            1        4              5    0            28      0       0  250   
4            1        4              5    0            28      0       0  250   

   proto   csum  ... code_icmp csum_icmp  port_no  rx_bytes_ave  rx_error_ave  \
0      1  32021  ...       160     62875        1            81             0   
1      1  63527  ...        32     63339        1            78             0   
2      1  45034  ...       236     62781        1            75             0   
3      1  27057  ...       165     63140        1            73             0   
4      1  62701  ...        68     63141        1            71             0   

   rx_dropped_a

In [ ]:
print("Test Data")
print(test_data.head())

Test Data
   datapath_id  version  header_length  tos  total_length  flags  offset  ttl  \
0            1        4              5    0            28      0       0  250   
1            1        4              5    0            28      0       0  250   
2            1        4              5    0            28      0       0  250   
3            1        4              5    0            28      0       0  250   
4            1        4              5    0            28      0       0  250   

   proto   csum  ... code_icmp csum_icmp  port_no  rx_bytes_ave  rx_error_ave  \
0      1  38973  ...       153     63162        1            42             0   
1      1   4617  ...       191     62936        1            42             0   
2      1   7768  ...       103     63132        1            42             0   
3      1   6176  ...       200     63149        1            42             0   
4      1  58402  ...       161     62960        1            42             0   

   rx_dropped_av

In [ ]:
def preprocess(data):
    columnsToEncode = ['src_ip', 'dst_ip']
    le = LabelEncoder()
    ohc = OneHotEncoder(handle_unknown='ignore')
    for feature in columnsToEncode:
        try:
            data[feature] = le.fit_transform(data[feature])
        except:
            print ('error ' + feature)
    return data

In [ ]:
preprocess(train_data)
train_df=pd.get_dummies(train_data, prefix=['label'])
preprocess(test_data)
test_df=pd.get_dummies(test_data, prefix=['label'])
train_df.head()

,datapath_id,version,header_length,tos,total_length,flags,offset,ttl,proto,csum,...,rx_dropped_ave,tx_bytes_ave,tx_error_ave,tx_dropped_ave,label_DDOS_ICMP,label_DDOS_TCP,label_DDOS_UDP,label_NORMAL_ICMP,label_NORMAL_TCP,label_NORMAL_UDP
0,1,4,5,0,28,0,0,250,1,32021,...,0,123,0,0,True,False,False,False,False,False
1,1,4,5,0,28,0,0,250,1,63527,...,0,123,0,0,True,False,False,False,False,False
2,1,4,5,0,28,0,0,250,1,45034,...,0,123,0,0,True,False,False,False,False,False
3,1,4,5,0,28,0,0,250,1,27057,...,0,123,0,0,True,False,False,False,False,False
4,1,4,5,0,28,0,0,250,1,62701,...,0,123,0,0,True,False,False,False,False,False


In [ ]:
y_train=train_df[["label_DDOS_ICMP","label_DDOS_TCP","label_DDOS_UDP","label_NORMAL_ICMP","label_NORMAL_TCP", "label_NORMAL_UDP"]].copy()
X_train=train_df.drop(["label_DDOS_ICMP","label_DDOS_TCP","label_DDOS_UDP","label_NORMAL_ICMP","label_NORMAL_TCP", "label_NORMAL_UDP"], axis=1)
y_test=test_df[["label_DDOS_ICMP","label_DDOS_TCP","label_DDOS_UDP","label_NORMAL_ICMP","label_NORMAL_TCP", "label_NORMAL_UDP"]].copy()
X_test=test_df.drop(["label_DDOS_ICMP","label_DDOS_TCP","label_DDOS_UDP","label_NORMAL_ICMP","label_NORMAL_TCP", "label_NORMAL_UDP"], axis=1)

print(X_test.shape)
print(X_train.shape)

(180000, 25)
(420000, 25)


In [ ]:
model = keras.Sequential([
    layers.Dense(25, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(25, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=['accuracy','CategoricalAccuracy']
)

In [ ]:
history = model.fit(X_train, y_train,epochs=100,batch_size=2100)

Epoch 1/100
200/200 [==============================] - 2s 10ms/step - loss: 3.4772 - accuracy: 0.8100 - categorical_accuracy: 0.8100
Epoch 2/100
200/200 [==============================] - 2s 10ms/step - loss: 0.1082 - accuracy: 0.9693 - categorical_accuracy: 0.9693
Epoch 3/100
200/200 [==============================] - 2s 8ms/step - loss: 0.0618 - accuracy: 0.9856 - categorical_accuracy: 0.9856
Epoch 4/100
200/200 [==============================] - 2s 9ms/step - loss: 0.0538 - accuracy: 0.9877 - categorical_accuracy: 0.9877
Epoch 5/100
200/200 [==============================] - 2s 8ms/step - loss: 0.0464 - accuracy: 0.9897 - categorical_accuracy: 0.9897
Epoch 6/100
200/200 [==============================] - 1s 6ms/step - loss: 0.3781 - accuracy: 0.9476 - categorical_accuracy: 0.9476
Epoch 7/100
200/200 [==============================] - 1s 6ms/step - loss: 3.9704 - accuracy: 0.7990 - categorical_accuracy: 0.7990
Epoch 8/100
200/200 [==============================] - 1s 6ms/step - loss:

In [ ]:
from sklearn.metrics import accuracy_score, mean_squared_error
from math import sqrt
from time import time

start = time()
y_pred=model.predict(X_test)
print(time() - start)
print(accuracy_score(np.round(y_pred),y_test))
print(sqrt(mean_squared_error(np.round(y_pred),y_test)))

5625/5625 [==============================] - 8s 1ms/step
10.403000354766846
0.9859722222222222
0.06837397147064098


Logistic Regression

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load the training dataset
train_data = pd.read_csv('/content/TRAIN-DATA.csv')

# Initialize label encoders for IP addresses and labels
ip_encoder = LabelEncoder()
label_encoder = LabelEncoder()

# Encode IP addresses
train_data['src_ip'] = ip_encoder.fit_transform(train_data['src_ip'])
train_data['dst_ip'] = ip_encoder.transform(train_data['dst_ip'])

# Encode labels
train_data['label'] = label_encoder.fit_transform(train_data['label'])

# Prepare feature matrix and labels
X = train_data.drop('label', axis=1)
y = train_data['label']

# Split the data into training and test sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Logistic Regression model
lr_model = LogisticRegression(max_iter=1000)  # Adjust max_iter if needed
lr_model.fit(X_train, y_train)

# Predict on the test set
y_pred_lr = lr_model.predict(X_test)

# Print the evaluation metrics
print("Logistic Regression Model Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_lr))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Model Accuracy: 0.9454166666666667
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13930
           1       1.00      1.00      1.00     13919
           2       1.00      1.00      1.00     14170
           3       0.84      0.85      0.84     13899
           4       0.84      0.83      0.84     14003
           5       0.98      1.00      0.99     14079

    accuracy                           0.95     84000
   macro avg       0.94      0.95      0.95     84000
weighted avg       0.95      0.95      0.95     84000



MLP Classifier

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the training dataset
train_data = pd.read_csv('/content/TRAIN-DATA.csv')

# Initialize label encoders for IP addresses and labels
ip_encoder = LabelEncoder()
label_encoder = LabelEncoder()

# Encode IP addresses
train_data['src_ip'] = ip_encoder.fit_transform(train_data['src_ip'])
train_data['dst_ip'] = ip_encoder.transform(train_data['dst_ip'])

# Encode labels
train_data['label'] = label_encoder.fit_transform(train_data['label'])

# Prepare feature matrix and labels
X = train_data.drop('label', axis=1)
y = train_data['label']

# Split the data into training and test sets (e.g., 80% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the MLP classifier
mlp_model = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=50, random_state=42)


In [ ]:
# Train the model on the training data
mlp_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(10,), max_iter=50, random_state=42)

In [ ]:
# Predict on the test set
y_pred_mlp = mlp_model.predict(X_test)

# Print the evaluation metrics
print("MLP Model Accuracy:", accuracy_score(y_test, y_pred_mlp))
print("MLP Classification Report:")
print(classification_report(y_test, y_pred_mlp))

MLP Model Accuracy: 0.7995357142857142
MLP Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     13930
           1       1.00      0.99      0.99     13919
           2       1.00      0.98      0.99     14170
           3       0.79      1.00      0.88     13899
           4       0.48      0.86      0.61     14003
           5       0.00      0.00      0.00     14079

    accuracy                           0.80     84000
   macro avg       0.71      0.80      0.74     84000
weighted avg       0.71      0.80      0.74     84000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
